In [1]:
import os
import shutil

from PIL import Image
import numpy as np

import nibabel as nib

from utils import read_tracking, write_tracking

In [2]:
main_path = "C:/Users/Bugra/Desktop/masters-thesis/liver_data"
views = ["l1", "l2", "l3", "r1", "r2", "r3"]

image_path = "C:/Users/Bugra/Desktop/masters-thesis/liver_all_sweeps_im.nii.gz"
confidence_path = "C:/Users/Bugra/Desktop/masters-thesis/liver_all_sweeps_conf_im.nii.gz"
tracking_path = "C:/Users/Bugra/Desktop/masters-thesis/liver_all_sweeps_tracking.csv"

views_path = os.path.join(main_path, "views")

In [3]:
# Read view trackings for deciding number of slices for each view
org_view_trackings = []
for view in views:
    org_view_trackings.append(read_tracking(os.path.join(main_path, view + ".ts")))

# Print shapes of views
for i, view in enumerate(views):
    print(f"{view} shape: {org_view_trackings[i].shape}")


l1 shape: (200, 4, 4)
l2 shape: (200, 4, 4)
l3 shape: (200, 4, 4)
r1 shape: (200, 4, 4)
r2 shape: (200, 4, 4)
r3 shape: (200, 4, 4)


In [4]:
# Read main image, confidence and tracking
image = nib.load(image_path).get_fdata().astype(np.uint8)
confidence = nib.load(confidence_path).get_fdata().astype(np.uint8)
tracking = read_tracking(tracking_path)

# Print max and min values of main image, confidence and tracking
print(f"image max: {np.max(image)}, min: {np.min(image)}")
print(f"confidence max: {np.max(confidence)}, min: {np.min(confidence)}")
print(f"tracking max: {np.max(tracking)}, min: {np.min(tracking)}")

# Print shapes of main image, confidence and tracking
print(f"image shape: {image.shape}")
print(f"confidence shape: {confidence.shape}")
print(f"tracking shape: {tracking.shape}")

# Split them into views, make sure to keep all metadata for nib files
view_images = []
view_confidences = []
view_trackings = []

start_idx = 0

for i, view in enumerate(views):

    end_idx = start_idx + org_view_trackings[i].shape[0]

    view_images.append(image[..., start_idx:end_idx])
    view_confidences.append(confidence[..., start_idx:end_idx])
    view_trackings.append(tracking[start_idx:end_idx])

    start_idx = end_idx


image max: 255, min: 0
confidence max: 255, min: 17
tracking max: 82.98933410644531, min: -48.94563674926758
image shape: (256, 516, 1200)
confidence shape: (256, 516, 1200)
tracking shape: (1200, 4, 4)


In [5]:
# Create new views directory
if os.path.exists(views_path):
    shutil.rmtree(views_path)
os.makedirs(views_path)

# Save views

for i, view in enumerate(views):
    view_image_path = os.path.join(views_path, view + "_im.nii.gz")
    view_confidence_path = os.path.join(views_path, view + "_conf_im.nii.gz")
    view_tracking_path = os.path.join(views_path, view + "_tracking.csv")

    nib.save(nib.Nifti1Image(view_images[i], np.eye(4)), view_image_path)
    nib.save(nib.Nifti1Image(view_confidences[i], np.eye(4)), view_confidence_path)
    write_tracking(view_tracking_path, view_trackings[i])

    print(f"Saved {view} image to {view_image_path}")
    print(f"Saved {view} confidence to {view_confidence_path}")
    print(f"Saved {view} tracking to {view_tracking_path}")


Saved l1 image to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\l1_im.nii.gz
Saved l1 confidence to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\l1_conf_im.nii.gz
Saved l1 tracking to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\l1_tracking.csv
Saved l2 image to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\l2_im.nii.gz
Saved l2 confidence to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\l2_conf_im.nii.gz
Saved l2 tracking to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\l2_tracking.csv
Saved l3 image to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\l3_im.nii.gz
Saved l3 confidence to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\l3_conf_im.nii.gz
Saved l3 tracking to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\l3_tracking.csv
Saved r1 image to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\r1_im.nii.gz
Saved r1 confidence to C:/Users/Bugra/Desktop/masters-thesis/liver_data\views\r1_conf_im.nii